# Biofuels Production

---

**Purpose:**

Data shaping & visualization

**Data Source:**

https://www.data.gov/

In [ ]:
# system libraries
import os
import sys

# check system information
print('Python Information', sys.version)
print('This is your current directory', os.getcwd())

In [ ]:
# datetime libraries
import datetime

# assgin current date and time
current_date = datetime.date.today()
current_time = datetime.datetime.now()

# check datetime information
print('Today is {}'.format(current_date))
print('Today is', datetime.datetime.strftime(current_date, '%m/%d/%Y'))
print('The time is', datetime.datetime.strftime(current_time, '%H:%M:%S'))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# pandas options
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 250
pd.options.display.max_columns = 50
pd.options.display.max_colwidth = 0
pd.options.display.precision = 2

import seaborn as sns

plt.style.use('bmh')

print(pd.__version__)
print(np.__version__)

In [ ]:
df = pd.DataFrame(
    {
        '':range(2000, 2011),
        'United States':(105.54, 115.71, 140.29, 183.87, 223.29, 260.61, 334.95, 457.33, 649.68, 746.52, 887.62),
        'Brazil':(183.89, 197.59, 216.93, 249.35, 251.72, 276.42, 307.31, 395.68, 486.35, 477.53, 527.32),
        'Germany':(4.3, 5.4, 8.8, 14, 20.4, 35.8, 59.4, 63.8 ,65, 58, 62),
        'India':(2.9, 3.0, 3.2, 3.3, 3.5, 3.9, 4.5, 4.7, 5.2, 7, 7),
        'Malawi':(.2,) * 11,
        'Australia':('--','-', 'NA', '', .10, .6, 1.7, 2.1, 3.4, 5.2, 7.9),   
    }
)


df.info()

In [ ]:
df.T

In [ ]:
fpath = "biofuels.csv"

df.T.to_csv(fpath, header=None)

## Data Preparation

In [ ]:
df = pd.read_csv(fpath)
df

In [ ]:
df.info()

In [ ]:
df = pd.read_csv(fpath, na_values=['-','--','NA'])
df.info()

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df.rename(columns={"Unnamed: 0":"Location"}, inplace=True)
df

In [ ]:
df.columns[1:].to_list()

In [ ]:
df = df.melt(
    id_vars=['Location'],
    value_vars=df.columns[1:].to_list(),
    var_name='Period',
    value_name='Production'
)

df.info()

In [ ]:
df.head(10)

In [ ]:
df.isna().sum()

In [ ]:
df.Production.fillna(0, inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
df['Year End'] = df['Period'].apply(lambda x: datetime.datetime(int(x),1,1))

df.info()

In [ ]:
df.head()

In [ ]:
countries = ["United States", "Brazil", "Germany", "India", "Malawi", "Australia"]

countries

In [ ]:
df = df[df['Location'].isin(countries)]
df

In [ ]:
df = df.pivot_table(index='Year End', columns='Location', values='Production', aggfunc=sum, fill_value=0)
df

In [ ]:
df.T

## Conditional Formatting

In [ ]:
df.style.highlight_max().format('{:,.4f}')

In [ ]:
df.style.highlight_max(axis=1).format("{:,.4f}")

In [ ]:
df.style.highlight_max(color='lime').highlight_min(color='pink').format("{:,.4f}")

In [ ]:
df.style.background_gradient('Blues').format("{:,.4f}")

In [ ]:
df.style.background_gradient('coolwarm').format("{:,.4f}")

In [ ]:
def check(val):
    if val < 20:
        return "background-color: orange"
    return ""

df.style.applymap(check).format('{:,.4f}')

## Line Charts

In [ ]:
df.plot();

In [ ]:
df.plot(marker='o');

In [ ]:
df.plot(
    subplots=True,
    lw=2,
    marker='s',
    alpha=0.7,
    layout=(2,3),
    figsize=(12,6),
    grid=True,
    rot=45,
    sharex=False
);

In [ ]:
for i, cols in df.iteritems():
    if cols.sum() > 3000:
        cols.plot(label=i, legend=True, marker='^', markerfacecolor='m')     

In [ ]:
df.sum(axis=1).plot(lw=3, marker='*');

## Area Charts

In [ ]:
df.plot.area();

In [ ]:
df.plot.area(stacked=False);

## Bar Charts

In [ ]:
picks=['Brazil','United States']

In [ ]:
df.assign(PERIOD = df.index.year)\
.plot.bar(x='PERIOD', y=picks);

In [ ]:
df.assign(PERIOD = df.index.year)\
.plot.barh(x='PERIOD', y=picks, stacked=True);

In [ ]:
df.sum().sort_values().plot.bar();

## Pie Charts

In [ ]:
pie_data = pd.Series(data=df.sum(), name='Biofuels Production')
pie_data

In [ ]:
pie_data.plot.pie(
    cmap='Spectral',
    figsize=(6,6),
    autopct='%.01f%%'
);

In [ ]:
pie_data[pie_data.values >200]

In [ ]:
pie_data = pie_data[pie_data.values >200]

pie_data.plot.pie(
    cmap='Spectral',
    figsize=(6,6),
    autopct='%.01f%%'
);

In [ ]:
pie_data.plot.pie(
    cmap='Spectral',
    autopct="%.1f%%",
    figsize=(6,6),
    pctdistance=0.9,
    wedgeprops=dict(width=0.3)
)

plt.show();

## Statistical Charts

In [ ]:
df.plot.box();

In [ ]:
df.plot.hist(subplots=True,figsize=(12, 6), layout=(2,3), sharex=False);

## Geospatial

In [ ]:
df = df.reset_index().melt(id_vars=['Year End'], var_name=['Location'], value_name='Production')
df

In [ ]:
!pip install geocoder

In [ ]:
import geocoder

In [ ]:
gdd = {}

for country in df['Location'].unique():
    gdd[country] = geocoder.osm(country).latlng

gdd

In [ ]:
!pip install folium --upgrade

In [ ]:
import folium

In [ ]:
gf = df.groupby('Location').sum().reset_index()
gf

In [ ]:
gf['LatLng'] = gf['Location'].map(gdd)
gf

In [ ]:
#Create base map
folium_map = folium.Map(location=(0, 0), zoom_start = 2, tiles = "CartoDB Positron")

#Plot Markers
for index, row in gf.iterrows():
    popup_string = "{}={:,.0f}".format(row['Location'], row['Production'])
    folium.Circle(
        location=row['LatLng'],
        popup=popup_string,
        radius=row['Production'] * 250,
        color='red' if row['Production'] > 1000 else 'blue',
        fill=True,
        fill_color='lightgrey',
        fill_opacity=.7,
        tooltip = row['Location'],
    ).add_to(folium_map)

# show the map
folium_map

## Trends & Forecasts

In [ ]:
xf = df.groupby(pd.Grouper(key='Year End', freq='y')).sum()
xf

In [ ]:
xf.plot();

In [ ]:
X = xf.index.year.values
X

In [ ]:
Y = xf.values.ravel()
Y

In [ ]:
np.corrcoef(X,Y)

In [ ]:
sns.regplot(X, Y);

In [ ]:
# numpy polyfit

# Creates a linear regression from the data points
m, b = np.polyfit(x=X, y=Y, deg=1)

print("m={:.02f},  b={:.02f}".format(m, b))

In [ ]:
# This is a simple y = mx + b line function
def f(x):
    return m*x + b

# This generates the same scatter plot as before, but adds a line plot using the function above
sns.scatterplot(x=X, y=Y, alpha=.4);
sns.lineplot(X, f(X), color='grey');

In [ ]:
fit = np.polyfit(x=X, y=Y, deg=1)

fit

In [ ]:
np.polyval(fit, np.arange(2011,2015))

In [ ]:
from sklearn.linear_model import LinearRegression

linreg = LinearRegression()

In [ ]:
linreg.fit(X.ravel().reshape(-1,1), Y.reshape(-1,1))

In [ ]:
print(linreg.intercept_)

print(linreg.coef_)

In [ ]:
X.ravel().reshape(-1,1)

In [ ]:
x_pred = np.arange(2011, 2015).reshape(-1,1)
x_pred

In [ ]:
y_pred = linreg.predict(x_pred)
y_pred

In [ ]:
X_new = np.concatenate((X, x_pred.ravel()))
X_new

In [ ]:
Y_new = np.concatenate((Y, y_pred.ravel()))
Y_new

In [ ]:
scenario = np.where(X_new <= X.max(), 'actual', 'forecast')
scenario

In [ ]:
sns.lineplot(x=X_new, y=Y_new, hue=scenario)
plt.scatter(x=X_new, y=Y_new, marker='o', s=50, c='gray', alpha=.7);
plt.xticks(rotation=45);